# Step 2 - Prepare network data
## Project: Growing Urban Bicycle Networks

This notebook downloads cycle, LTN, and street network data sets and prepares them for analysis.

In [ ]:
# import libraries
from src import utils
PATH = utils.PATH # shortening the var name so that we don't have to change it below

import os
import yaml
import json
import pprint
pp = pprint.PrettyPrinter(indent=4)

import numpy as np
import networkx as nx

import matplotlib.pyplot as plt
from matplotlib import cm

from tqdm.notebook import tqdm

import osmnx as ox
ox.settings.log_file = True
ox.settings.requests_timeout = 300
ox.settings.logs_folder = PATH["logs"]
import fiona
import shapely
from shapely.geometry import Polygon

### Parameters

In [ ]:
debug=False

In [ ]:
params = yaml.load(
    open("../parameters/parameters.yml"), 
    Loader=yaml.FullLoader)
osmnxparameters = json.load(open("../parameters/osmnxparameters.json", "r"))
plotparam = json.load(open("../parameters/plotparam.json", "r"))
plotparam_analysis = json.load(open("../parameters/plotparam_analysis.json", "r"))

## Download and wrangle data

In [ ]:
# load cities
cities = utils.load_cities(PATH, debug)

# create city subfolders
utils.create_city_subfolders(PATH, cities)

### Networks

In [ ]:
for scenario in params["scenarios"]:
    for placeid, placeinfo in tqdm(cities.items(), desc=f"Cities ({scenario})"):
        base_path = os.path.join(PATH["data"], placeid, scenario)

        if placeinfo["nominatimstring"] != '':
            location = ox.geocoder.geocode_to_gdf(placeinfo["nominatimstring"])
            if location.geometry[0].geom_type == 'MultiPolygon':
                location = location.explode(index_parts=False).reset_index(drop=True)
            location = utils.fill_holes(
                utils.extract_relevant_polygon(
                    placeid, shapely.geometry.shape(location['geometry'][0])))
            if debug:
                try:
                    color = cm.rainbow(np.linspace(0, 1, len(location)))
                    for poly, c in zip(location, color):
                        plt.plot(*poly.exterior.xy, c=c)
                        for intr in poly.interiors:
                            plt.plot(*intr.xy, c="red")
                except:
                    plt.plot(*location.exterior.xy)
                plt.show()
        else:
            shp = fiona.open(PATH["data"] + placeid + "/" + placeid + ".shp")
            first = next(iter(shp))
            try:
                location = Polygon(shapely.geometry.shape(first['geometry']))
            except:
                location = shapely.geometry.shape(first['geometry'])

        Gs = {}
        for parameterid, parameterinfo in tqdm(osmnxparameters.items(), desc="Networks", leave=False):
            for i in range(10):
                try:
                    Gs[parameterid] = ox.graph_from_polygon(
                        location,
                        network_type=parameterinfo['network_type'],
                        custom_filter=parameterinfo['custom_filter'],
                        retain_all=parameterinfo['retain_all'],
                        simplify=False
                    )
                    break
                except ValueError:
                    Gs[parameterid] = nx.empty_graph(create_using=nx.MultiDiGraph)
                    print(placeid + ": No OSM data for graph " + parameterid + ". Created empty graph.")
                    break
                except (ConnectionError, UnboundLocalError):
                    print("ConnectionError or UnboundLocalError. Retrying.")
                    continue
                except:
                    print("Other error. Retrying.")
                    continue
            if parameterinfo['export']:
                utils.ox_to_csv(Gs[parameterid], base_path + "/", placeid, parameterid)
                ox.save_graph_geopackage(Gs[parameterid], filepath=base_path + "/" + placeid + "_" + parameterid + ".gpkg", directed=False)

        # Load and process LTN neighbourhoods (skip for no_ltn_scenario)
        city_neighbourhood_streets = {}
        if scenario != "no_ltn_scenario":
            neighbourhoods = utils.load_neighbourhoods(base_path + "/")
            if not neighbourhoods:
                print(placeid + ": No LTN dataset found.")
            else:
                neighbourhoods = utils.prepare_neighbourhoods(neighbourhoods)
                for city_name, gdf in neighbourhoods.items():
                    if debug:
                        print(f"Processing streets for {city_name}...")
                    # Patch cycle graphs with pedestrian links and get results
                    results = utils.patch_cycle_graph_with_pedestrian_links(neighbourhoods, gdf, debug=debug)
                    for city_name, stats in results.items():
                        patched_edges = stats.get("patched_edges")
                        if not patched_edges.empty:
                            patched_edges.to_file(os.path.join(PATH["data"], placeid, scenario, f"{placeid}_patched_edges.gpkg"), driver='GPKG'),
                        nodes, edges = ox.graph_to_gdfs(stats['cycle_graph_after'])
                        city_neighbourhood_streets[city_name] = {
                            'nodes': nodes,
                            'edges': edges,
                            'neighbourhood_graphs': stats['cycle_graph_after']}
                    # Plot and save patching stats & CSV
                    output_plot_path = os.path.join(PATH["plots"] + "/" + placeid + "/" + scenario + "/" + f"{placeid}_pedestrian_paths_in_ltn_analysis.png")
                    output_csv_path = os.path.join(PATH["results"], placeid, scenario, "pedestrian_in_ltn.csv")
                    utils.plot_and_save_network_stats(results, output_plot_path, output_csv_path, scenario)

        # Compose graphs
        if scenario != "no_ltn_scenario" and city_name in city_neighbourhood_streets:
            neighbourhood_graph = city_neighbourhood_streets[city_name]['neighbourhood_graphs']
            neighbourhood_graph = utils.clean_edge_attributes(neighbourhood_graph)
            Gs['biketrack'] = nx.compose_all([
                Gs['bike_cyclewaylefttrack'], Gs['bike_cyclewaytrack'],
                Gs['bike_highwaycycleway'], Gs['bike_bicycleroad'],
                Gs['bike_cyclewayrighttrack'], Gs['bike_designatedpath'],
                Gs['bike_cyclestreet'], neighbourhood_graph
            ])
        else:
            Gs['biketrack'] = nx.compose_all([
                Gs['bike_cyclewaylefttrack'], Gs['bike_cyclewaytrack'],
                Gs['bike_highwaycycleway'], Gs['bike_bicycleroad'],
                Gs['bike_cyclewayrighttrack'], Gs['bike_designatedpath'],
                Gs['bike_cyclestreet']
            ])

        parameterid = 'biketrack'
        utils.ox_to_csv(Gs[parameterid], base_path + "/", placeid, parameterid)
        ox.save_graph_geopackage(Gs[parameterid], filepath=base_path + "/" + placeid + "_" + parameterid + ".gpkg", directed=False)

        parameterid = 'bikeable'
        Gs[parameterid] = nx.compose_all([Gs['biketrack'], Gs['car30'], Gs['bike_livingstreet']])
        utils.ox_to_csv(Gs[parameterid], base_path + "/", placeid, parameterid)
        ox.save_graph_geopackage(Gs[parameterid], filepath=base_path + "/" + placeid + "_" + parameterid + ".gpkg", directed=False)

        parameterid = 'biketrackcarall'
        Gs[parameterid] = nx.compose(Gs['biketrack'], Gs['carall'])
        utils.ox_to_csv(Gs[parameterid], base_path + "/", placeid, parameterid)
        ox.save_graph_geopackage(Gs[parameterid], filepath=base_path + "/" + placeid + "_" + parameterid + ".gpkg", directed=False)

        parameterid = 'biketrack_no_ltn'
        Gs[parameterid] = nx.compose_all([
            Gs['bike_cyclewaylefttrack'], Gs['bike_cyclewaytrack'],
            Gs['bike_highwaycycleway'], Gs['bike_bicycleroad'],
            Gs['bike_cyclewayrighttrack'], Gs['bike_designatedpath'],
            Gs['bike_cyclestreet']
        ])
        utils.ox_to_csv(Gs[parameterid], base_path + "/", placeid, parameterid)
        ox.save_graph_geopackage(Gs[parameterid], filepath=base_path + "/" + placeid + "_" + parameterid + ".gpkg", directed=False)

        for parameterid in params["networktypes"][:-2]:
            utils.ox_to_csv(ox.simplify_graph(Gs[parameterid]), base_path + "/", placeid, parameterid, "_simplified")


In [ ]:
# Compress all data files (will not do anything if files were compressed already)
for folder, subfolders, files in os.walk(PATH["data"]):
    for file in files:
        if file.endswith('es.csv'):
            utils.compress_file(folder + "/", file.split(".")[0])